Fine-tune a Pre-trained Model Using HuggingFace Transformers
Fine-tuning a pretrained model allows you to leverage the vast amount of knowledge encoded in the model from its initial training on large datasets. This approach significantly reduces the time and computational resources required compared to training a model from scratch. It also helps achieve high performance with relatively small amounts of task-specific data, making it a powerful technique in machine learning and AI development.

Steps for Fine-Tuning a Pretrained Model
Choose a Pretrained Model:
Select a model from the Hugging Face Model Hub that suits your task. For example, if you're working on text classification, models like BERT or RoBERTa are popular choices.

Prepare Your Dataset:
Ensure your dataset is properly formatted. For text tasks, this usually involves tokenizing your text data. You can use the Tokenizer provided by the Transformers library to convert your text into input IDs and attention masks.

Set Up Training Arguments:
Define your training parameters using TrainingArguments. This includes specifying the output directory, evaluation strategy, learning rate, batch size, and number of epochs.

Create a Trainer:
Instantiate a Trainer object, which will handle the training process. You need to provide your model, training arguments, training dataset, evaluation dataset, and a function to compute metrics.

Train the Model:
Call the train() method on your Trainer object to start the fine-tuning process.

In [4]:
!pip install transformers datasets evaluate accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


You’ll also need to install your preferred machine learning framework - Pytorch or TensorFlow.


In [5]:
!pip install torch


In [7]:
# prompt: hugging login by token

import os

from huggingface_hub import login

# Replace with your actual token
token = ""

# Log in to Hugging Face Hub
login(token=token)

# You can verify the login by checking the environment variable
print(f"Hugging Face Token: {os.environ.get('HF_TOKEN')}")

Hugging Face Token: None


Begin by loading the Yelp Reviews dataset:

In [8]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

Tokenize the text data to prepare it for the model.

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

If you like, you can create a smaller subset of the full dataset to fine-tune on to reduce the time it takes:

In [10]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Train with PyTorch Trainer

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training hyperparameters
create a TrainingArguments class which contains all the hyperparameters you can tune as well as flags for activating different training options.

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

Evaluate
Trainer does not automatically evaluate model performance during training. You’ll need to pass Trainer a function to compute and report metrics.

In [13]:
!pip install scikit-learn

In [14]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [17]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [27]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [32]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    logging_dir="./logs",
    report_to="none",  # Disables wandb integration
)


Trainer
Create a Trainer object with your model, training arguments, training and test datasets, and evaluation function.

In [35]:
# In cell 17, where the Trainer is initially defined:
from transformers import Trainer, TrainingArguments

# Define training_args once
training_args = TrainingArguments(
    output_dir="./results",  # Set output directory here
    logging_dir="./logs",  # Set logging directory here
    report_to="none",  # Disables wandb integration
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

# Remove cells 27 and 32 where training_args is redefined.

In [36]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=375, training_loss=0.1670426025390625, metrics={'train_runtime': 293.8218, 'train_samples_per_second': 10.21, 'train_steps_per_second': 1.276, 'total_flos': 789354427392000.0, 'train_loss': 0.1670426025390625, 'epoch': 3.0})

In [37]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 3.1012134552001953, 'eval_accuracy': 0.531, 'eval_runtime': 28.966, 'eval_samples_per_second': 34.523, 'eval_steps_per_second': 4.315, 'epoch': 3.0}
